In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
import jax
import jax.flatten_util
from tracr.rasp import rasp
from tracr.compiler import basis_inference
from tracr.compiler import craft_graph_to_model
from tracr.compiler import expr_to_craft_graph
from tracr.compiler import rasp_to_graph
from tracr.craft import bases
from tracr.rasp import rasp
import networkx as nx

from rasp_generator import utils, sampling, map_primitives
from rasp_tokenizer import tokenizer

from rasp_generator.utils import FunctionWithRepr

In [2]:
sampler = sampling.ProgramSampler()
sampler.sample()
model, tokens, params = tokenizer.compile_and_tokenize(sampler.program)

In [6]:
jax.tree_map(lambda x: x.shape, params)

{'layer_0/mlp': (3446,),
 'layer_1/attn': (3688,),
 'layer_2/mlp': (3446,),
 'layer_3/attn': (3688,)}

In [5]:
# sop_1 = rasp.Map(lambda x: x + 1, rasp.indices)
# sel_2 = rasp.Select(sop_1, rasp.tokens, rasp.Comparison.EQ)
# sop_3 = rasp.Aggregate(sel_2, sop_1)
# sop_4 = rasp.SequenceMap(lambda x, y: x + y, sop_1, sop_3)
# sop_5 = rasp.SelectorWidth(sel_2)
# sop_6 = rasp.SequenceMap(lambda x, y: x + y, sop_3, sop_5)

sop_1 = rasp.Map(FunctionWithRepr("lambda x: x + 1"), rasp.tokens)
sel_2 = rasp.Select(sop_1, rasp.tokens, rasp.Comparison.EQ)
sop_3 = rasp.Aggregate(sel_2, sop_1)
#sop_4 = rasp.SequenceMap(
#    FunctionWithRepr("lambda x, y: x * y"), sop_1, sop_3)

program = sop_3